# This file will reshape the shift dataset as follows:

We want one row for each shift, and some ~500 columns for each NBA player. All values will be zero except for the 10 players that participate in the given shift. 

In [6]:
import pandas as pd
import numpy as np

shifts = pd.read_csv("../data/shifts_data_2018_19.csv")
shifts.head()

,Unnamed: 0,point_differential,num_possessions,home_team,away_team,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5
0,0,-7,19.2000,Celtics,Nuggets,202694,1628369,201143,202681,203935,1627750,203914,203115,200794.0,203999.0
1,1,3,7.6800,Celtics,Nuggets,1628369,203382,201143,202681,203935,1627750,203914,203115,200794.0,203999.0
2,2,-1,1.3824,Celtics,Nuggets,1627759,203382,201143,202681,203935,1627750,203914,203115,200794.0,203999.0
3,3,-1,2.7648,Celtics,Nuggets,1627759,203382,201143,202681,203935,1628420,203914,203115,200794.0,203999.0
4,4,2,5.2224,Celtics,Nuggets,1627759,203382,201143,202681,203935,1628420,203115,1627736,203486.0,203999.0


In [16]:
shifts.dropna(subset=['away_player_5'], how='all', inplace=True)
shifts.dropna(subset=['away_player_4'], how='all', inplace=True)
shifts.dropna(subset=['away_player_3'], how='all', inplace=True)
shifts.dropna(subset=['away_player_2'], how='all', inplace=True)
shifts.dropna(subset=['away_player_1'], how='all', inplace=True)

,Unnamed: 0,point_differential,num_possessions,home_team,away_team,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5
0,0,-7,19.2000,Celtics,Nuggets,202694,1628369,201143,202681,203935,1627750,203914,203115,200794.0,203999.0
1,1,3,7.6800,Celtics,Nuggets,1628369,203382,201143,202681,203935,1627750,203914,203115,200794.0,203999.0
2,2,-1,1.3824,Celtics,Nuggets,1627759,203382,201143,202681,203935,1627750,203914,203115,200794.0,203999.0
3,3,-1,2.7648,Celtics,Nuggets,1627759,203382,201143,202681,203935,1628420,203914,203115,200794.0,203999.0
4,4,2,5.2224,Celtics,Nuggets,1627759,203382,201143,202681,203935,1628420,203115,1627736,203486.0,203999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33885,33885,-2,3.8400,Bulls,Pacers,201152,202083,202709,1626167,202711,203490,201577,1627853,1628374.0,203897.0
33886,33886,7,18.0096,Bulls,Pacers,201152,201954,202083,1626167,202711,203490,201577,1628374,203897.0,1627739.0
33887,33887,2,3.3024,Bulls,Pacers,201152,201954,202083,1626167,202711,203490,1627782,1628374,203897.0,1627739.0
33888,33888,-1,1.3824,Bulls,Pacers,201152,201954,202083,1626167,202711,203490,201577,1628374,203897.0,1627739.0


## Method:

We will construct a dictionary with the key = player_id and the value = the index at which this player will be kept in the final dataframe. So we will get all unique players for each column ('home_player_1', 'home_player_2', etc.) and iterate through these unique lists checking if a player is already in the dict - if so, continue, if not, add player to the dict and increment index.

Once we have this dictionary, we can initialize a player matrix with N rows and M columns where N is the number of shifts and M is the number of unique players, and we initialize this matrix with all zeros. Then we iterate through the original shifts dataset and specifically inspect each of the player columns, and for each player we see we get his index via the dictionary created above and we set that index at that row equal to 1. This may be computationally intensive but should be feasible.

In [17]:
# This code block makes the player_index mapping

index = 0 # initialize index to zero
player_index_map = dict() # initialize player to index mapping as dictionary
# first iterate through home columns (this should cover everything, but we will also do away columns just in case)
for i in range(1, 6):
    name = "home_player_" + str(i)
    players = pd.unique(shifts[name])
    for player in players:
        if player not in player_index_map:
            player_index_map[player] = index
            index += 1
        else:
            continue # if player is already in the index then continue
            
# now iterate through away columns to catch any special cases of players who only participated in away games
for i in range(1,6):
    name = "away_player_" + str(i)
    players = pd.unique(shifts[name])
    for player in players:
        if player not in player_index_map:
            player_index_map[player] = index
            index += 1
        else:
            continue # if player is already in the index then continue

In [18]:
M =len(player_index_map.keys()) # 531 unique NBA players in 2018-19 season according to our data. This seems reasonable
M

529

In [19]:
N = shifts.shape[0] # number of unique shifts in 2018-19 season
players_mat = np.zeros((N,M)) # initialize matrix

for i in range(N):
    # first handle home players
    for j in range(1,6):
        name = "home_player_" + str(j)
        player_id = shifts[name].iloc[i]
        index = player_index_map[player_id]
        players_mat[i, index] = 1
    # Now handle away players
    for j in range(1,6):
        name = "away_player_" + str(j)
        player_id = shifts[name].iloc[i]
        index = player_index_map[player_id]
        # set away players to -1
        players_mat[i, index] = -1
    

In [22]:
players_df = pd.DataFrame(players_mat)
point_diff = shifts['point_differential']
num_poss = shifts['num_possessions']
point_diff_per_100 = point_diff / num_poss
home_team = shifts['home_team']
away_team = shifts['away_team']
# point_diff_per_100
# players_df.insert(loc = 0, column = "point_differential", value = point_diff)
# players_df.insert(loc = 1, column = "num_possessions", value = num_poss)
players_df.insert(loc = 0, column = "point_diff_per_100", value = point_diff_per_100)
players_df.insert(loc = 1, column = "home_team", value = home_team)
players_df.insert(loc = 2, column = "away_team", value = away_team)
players_df # This is the final dataframe that we will use to train our model

,point_diff_per_100,home_team,away_team,0,1,2,3,4,5,6,...,519,520,521,522,523,524,525,526,527,528
0,-0.364583,Celtics,Nuggets,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.390625,Celtics,Nuggets,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.723380,Celtics,Nuggets,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.361690,Celtics,Nuggets,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.382966,Celtics,Nuggets,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33885,-0.520833,Bulls,Pacers,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33886,0.388682,Bulls,Pacers,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33887,0.605620,Bulls,Pacers,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33888,-0.723380,Bulls,Pacers,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
player_index_df = pd.DataFrame(list(player_index_map.items()))
player_index_df.columns = ["player_id", "index"]
player_index_df
player_index_df.to_csv(r'../data/player_index_map.csv')

In [24]:
players_df.to_csv(r'../data/shifts_data_final_2018_19.csv')